In [1]:
import pandas as pd
sku_sales = pd.read_csv("../data/sku_sales.csv")
# sku_prom = pd.read_csv("../data/sku_prom.csv")
sku_price_and_status = pd.read_csv("../data/sku_price_and_status.csv")

In [2]:
sku_sales = sku_sales.rename(columns={'order_time':'date'})
sku_sales['date'] = pd.to_datetime(sku_sales['date']).dt.strftime('%Y-%m-%d')

In [3]:
sku_sales_with_price = pd.merge(sku_sales[['order_id', 'date', 'store_id', 'sku_id', 'quantity', 'channel']], 
                                  sku_price_and_status[['date', 'store_id', 'sku_id', 'original_price']], 
                                  on=['date', 'store_id', 'sku_id'], how='left')

In [4]:
sku_sales_with_price['original_amount'] = sku_sales_with_price['quantity'] * sku_sales_with_price['original_price']

In [6]:
sku_sales_with_price.sort_values(by=['order_id', 'sku_id', 'channel'], ascending=[True, True, True,], inplace=True)
sku_sales_with_price = sku_sales_with_price[['order_id', 'date', 'store_id', 'sku_id', 'channel', 'quantity', 'original_price', 'original_amount']]

In [7]:
sku_sales_with_price.to_csv("sku_sales_with_price.csv", index = False)

---

---

In [1]:
import pandas as pd
from pandarallel import pandarallel
df1 = pd.read_csv("./sku_sales_with_price.csv")
df2 = pd.read_csv("../data/sku_prom.csv")

In [2]:
pandarallel.initialize(progress_bar=True)
order_amounts = df1.groupby(['order_id', 'date', 'store_id', 'channel'])['original_amount'].sum().reset_index()
# 计算各订单的总金额

INFO: Pandarallel will run on 24 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [3]:
def calculate_discounted_amount(row):
    order_id = row['order_id']
    date = row['date']
    store_id = row['store_id']
    channel = row['channel']
    order_amount = order_amounts[(order_amounts['order_id'] == order_id) & (order_amounts['date'] == date) & (order_amounts['store_id'] == store_id) & (order_amounts['channel'] == channel)]['original_amount'].values[0]
    sku_id = row['sku_id']
    promo_info = df2[(df2['store_id'] == store_id) & (df2['sku_id'] == sku_id) & (df2['date'] == date) & (df2['channel'] == channel)]
    if not promo_info.empty:
        threshold = promo_info['threshold'].values[0]
        discount_off = promo_info['discount_off'].values[0]
        original_amount = row['original_amount']
        if order_amount > threshold * original_amount:
            return original_amount * (1 - discount_off)
    
    return row['original_amount']  # 没有促销信息或不满足条件，原价
    
df1['discounted_amount'] = df1.parallel_apply(calculate_discounted_amount, axis=1)


Process ForkPoolWorker-21:
Process ForkPoolWorker-19:
Process ForkPoolWorker-25:
Process ForkPoolWorker-17:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/root/miniconda3/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/root/miniconda3/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/root/miniconda3/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/root/miniconda3/lib/python3.8/multiprocessing/pool.py", line 125, in worker
    result = (True, func(*args, **kwds))
  File "/root/miniconda3/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "/root/miniconda3/lib/python3.8/multiprocessing/pool.py", line 51, in starmapstar
    return list(itertools.starmap(args[0], args[1]))
Traceback (most recent call last):
  File "

KeyboardInterrupt: 

  File "/root/miniconda3/lib/python3.8/site-packages/pandarallel/core.py", line 95, in __call__
    result = self.work_function(
  File "/root/miniconda3/lib/python3.8/site-packages/pandas/core/apply.py", line 678, in apply
    return self.apply_standard()
  File "/root/miniconda3/lib/python3.8/site-packages/pandas/core/apply.py", line 678, in apply
    return self.apply_standard()
  File "/root/miniconda3/lib/python3.8/multiprocessing/pool.py", line 51, in starmapstar
    return list(itertools.starmap(args[0], args[1]))
  File "/root/miniconda3/lib/python3.8/site-packages/pandarallel/data_types/dataframe.py", line 32, in work
    return data.apply(
  File "/root/miniconda3/lib/python3.8/site-packages/pandas/core/ops/array_ops.py", line 293, in comparison_op
    res_values = comp_method_OBJECT_ARRAY(op, lvalues, rvalues)
  File "/root/miniconda3/lib/python3.8/site-packages/pandas/core/apply.py", line 798, in apply_standard
    results, res_index = self.apply_series_generator()
Keyboard